In [29]:
import pandas as pd
import numpy as np
import progressbar

In [2]:
df = pd.read_csv("fer2013.csv")

In [33]:
df.shape

(35887, 3)

In [37]:
f = open("fer2013.csv")
f.__next__()
trainImage, trainLables = [],[]
valImage, valLables = [],[]
testImage, testLables = [],[]

In [38]:
widgets = ["Image Processing", progressbar.Percentage(), " ", progressbar.Bar(), " ", progressbar.ETA()]
pbar = progressbar.ProgressBar(maxval=35887, widgets=widgets).start()

for n, i in enumerate(f):
    lable, image, uses = i.strip().split(",")
    image = np.array(image.split(" "), dtype = "uint8")
    image = image.reshape((48,48, 1))
    if uses == "Training":
        trainImage.append(image)
        trainLables.append(lable)
    elif uses == "PrivateTest":
        valImage.append(image)
        valLables.append(lable)
    else:
        testImage.append(image)
        testLables.append(lable)
    pbar.update(n)
pbar.finish()
        
        
        
        

Image Processing100% |##########################################| Time: 0:00:21


In [52]:
trainImage = np.array(trainImage)
valImage = np.array(valImage)
testImage = np.array(testImage)

In [39]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import  Conv2D, MaxPooling2D, Flatten, Dense, Dropout

from tensorflow.keras.preprocessing.image import img_to_array, load_img, ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from imutils import paths
import numpy as np
import cv2
import os
import progressbar

In [40]:
def LeNet(width, height, depth, classes):
    model = Sequential()

    model.add(Conv2D(20, (5,5), activation = "relu", input_shape = (width, height, depth)))
    model.add(MaxPooling2D(pool_size = (2,2), strides = (2,2)))


    model.add(Conv2D(50, (5,5), activation = "relu"))
    model.add(MaxPooling2D(pool_size = (2,2), strides = (2,2)))

    model.add(Flatten())
    model.add(Dense(500, activation = "relu"))

    model.add(Dense(classes, activation = "softmax"))

    return model

In [41]:
augmentation = ImageDataGenerator(rotation_range=30, width_shift_range=0.1, 
                                  height_shift_range=0.1, zoom_range=0.2,
                                  horizontal_flip=True, fill_mode="nearest")

In [47]:
from sklearn.preprocessing import LabelBinarizer

trainLabels = LabelBinarizer().fit_transform(trainLables)
valLabels = LabelBinarizer().fit_transform(valLables)
testLabels = LabelBinarizer().fit_transform(testLables)


In [ ]:

model = LeNet(48, 48, 1, len(trainLabels[0]))
model.compile(loss = "binary_crossentropy", optimizer="adam", metrics=["accuracy"])
print("Model is Going to Train.....")
H = model.fit_generator(augmentation.flow(trainImage, trainLabels, batch_size = 32), validation_data=(valImage, valLabels), steps_per_epoch=len(trainImage)//32, epochs=10, verbose=1)             


In [65]:
#### Live Emotion Prediction/


Emotion = ["Angry", "Disgust", "Fear", "Happy", "Sad", "Surprise", "Neutral"]
    